In [3]:
import time
from PIL import Image
import numpy as np
from loguru import logger

def analyze_image_loading(image_path):
    # Measure image reading time
    start_time = time.time()
    img = Image.open(image_path)
    read_time = time.time() - start_time
    
    # Get original image dimensions
    original_width, original_height = img.size
    
    # Measure convert time
    start_time = time.time()
    img_rgb = img.convert("RGB")
    convert_time = time.time() - start_time
    
    # Measure numpy array conversion time
    start_time = time.time()
    # img_array = np.array(img_rgb)
    numpy_time = time.time() - start_time
    
    # Get final array dimensions
    # array_height, array_width = img_array.shape[:2]
    
    # Log results
    logger.info(f"Image Path: {image_path}")
    logger.info(f"Original Dimensions: {original_width}x{original_height}")
    # logger.info(f"Array Dimensions: {array_width}x{array_height}")
    logger.info(f"Time Analysis:")
    logger.info(f"  - Image.open(): {read_time:.3f} seconds")
    logger.info(f"  - convert('RGB'): {convert_time:.3f} seconds")
    logger.info(f"  - np.array(): {numpy_time:.3f} seconds")
    logger.info(f"  - Total time: {read_time + convert_time + numpy_time:.3f} seconds")
    
    # Memory cleanup
    img.close()
    del img_rgb
    # del img_array

if __name__ == "__main__":
    # image_path = "/data2/zly/mot_data/crowdhuman/Images/273278,68170000a311a0cd.jpg"
    image_path = "/data2/zly/mot_data/crowdhuman/Images/273275,9ea9000f265093f.jpg"
    # image_path = "/data3/zly/crowdhuman/Images/273278,68170000a311a0cd.jpg"
    analyze_image_loading(image_path)

2024-12-13 17:34:07.606 | INFO     | __main__:analyze_image_loading:29 - Image Path: /data2/zly/mot_data/crowdhuman/Images/273275,9ea9000f265093f.jpg
2024-12-13 17:34:07.608 | INFO     | __main__:analyze_image_loading:30 - Original Dimensions: 4032x3024
2024-12-13 17:34:07.609 | INFO     | __main__:analyze_image_loading:32 - Time Analysis:
2024-12-13 17:34:07.609 | INFO     | __main__:analyze_image_loading:33 -   - Image.open(): 0.003 seconds
2024-12-13 17:34:07.610 | INFO     | __main__:analyze_image_loading:34 -   - convert('RGB'): 0.322 seconds
2024-12-13 17:34:07.611 | INFO     | __main__:analyze_image_loading:35 -   - np.array(): 0.000 seconds
2024-12-13 17:34:07.612 | INFO     | __main__:analyze_image_loading:36 -   - Total time: 0.325 seconds


In [4]:

def analyze_image_loading(image_path):
    # 测试不同的转换方法
    
    # 方法1：标准方法
    img = Image.open(image_path)
    img_rgb = img.convert("RGB")
    # start_time = time.time()
    # img_array1 = np.array(img_rgb)
    # time1 = time.time() - start_time
    # logger.info(f"Standard np.array(): {time1:.3f} seconds")
    
    # 方法2：使用asarray（避免数据复制）
    start_time = time.time()
    img_array2 = np.asarray(img_rgb)
    time2 = time.time() - start_time
    logger.info(f"np.asarray(): {time2:.3f} seconds")
    
    # 方法3：通过frombuffer
    start_time = time.time()
    img_array3 = np.frombuffer(img_rgb.tobytes(), dtype=np.uint8)
    img_array3 = img_array3.reshape((img_rgb.size[1], img_rgb.size[0], 3))
    time3 = time.time() - start_time
    logger.info(f"np.frombuffer(): {time3:.3f} seconds")
    
    # # 打印内存信息
    # logger.info(f"Image shape: {img_array1.shape}")
    # logger.info(f"Memory per array: {img_array1.nbytes / 1024 / 1024:.2f} MB")
    
    # 清理内存
    # del img_array1
    del img_array2
    del img_array3
    img.close()
analyze_image_loading(image_path)

2024-12-13 17:34:11.209 | INFO     | __main__:analyze_image_loading:16 - np.asarray(): 0.057 seconds
2024-12-13 17:34:11.267 | INFO     | __main__:analyze_image_loading:23 - np.frombuffer(): 0.057 seconds


In [6]:
import time
from PIL import Image
import numpy as np
from loguru import logger

def test_array_modifications(image_path):
    # 读取图片
    img = Image.open(image_path)
    img_rgb = img.convert("RGB")
    
    def create_array():
        return np.array(img_rgb)
        
    def create_asarray():
        return np.asarray(img_rgb).copy()  # 确保可写
        
    def create_frombuffer():
        # 创建可写的数组
        return np.frombuffer(img_rgb.tobytes(), dtype=np.uint8).reshape((img_rgb.size[1], img_rgb.size[0], 3)).copy()
    
    def modify_array(arr):
        # 验证数组是否可写
        if arr.flags.writeable:
            arr[..., 0] += 10  # 修改红色通道
        else:
            logger.warning("Array is read-only!")
    
    # 测试不同方法的创建+修改时间
    def time_operation(name, creation_func, modification_func):
        # 测试创建时间
        start_time = time.time()
        arr = creation_func()
        create_time = time.time() - start_time
        
        # 打印数组信息
        logger.info(f"\n{name} array info:")
        logger.info(f"Writeable: {arr.flags.writeable}")
        logger.info(f"Shape: {arr.shape}")
        
        # 测试修改时间
        start_time = time.time()
        modification_func(arr)
        modify_time = time.time() - start_time
        
        logger.info(f"\n{name} timing:")
        logger.info(f"Creation time: {create_time:.3f}s")
        logger.info(f"Modification time: {modify_time:.3f}s")
        logger.info(f"Total time: {create_time + modify_time:.3f}s")
        return arr
    
    # 测试三种方法
    # 1. np.array
    arr1 = time_operation(
        "np.array",
        create_array,
        modify_array
    )
    
    # 2. np.asarray
    arr2 = time_operation(
        "np.asarray",
        create_asarray,
        modify_array
    )
    
    # 3. np.frombuffer
    arr3 = time_operation(
        "np.frombuffer",
        create_frombuffer,
        modify_array
    )
    
    # 验证结果是否相同
    logger.info("\nVerifying results:")
    logger.info(f"All arrays modified equally: {np.array_equal(arr1, arr2) and np.array_equal(arr1, arr3)}")
    
    # 清理内存
    del arr1, arr2, arr3
    img.close()

if __name__ == "__main__":
    image_path = "/data2/zly/mot_data/crowdhuman/Images/273278,68170000a311a0cd.jpg"
    test_array_modifications(image_path)

2024-12-13 17:42:54.396 | INFO     | __main__:time_operation:36 - 
np.array array info:
2024-12-13 17:42:54.397 | INFO     | __main__:time_operation:37 - Writeable: True
2024-12-13 17:42:54.397 | INFO     | __main__:time_operation:38 - Shape: (2316, 3970, 3)
2024-12-13 17:42:54.405 | INFO     | __main__:time_operation:45 - 
np.array timing:
2024-12-13 17:42:54.406 | INFO     | __main__:time_operation:46 - Creation time: 57.069s
2024-12-13 17:42:54.406 | INFO     | __main__:time_operation:47 - Modification time: 0.007s
2024-12-13 17:42:54.407 | INFO     | __main__:time_operation:48 - Total time: 57.076s
2024-12-13 17:42:54.430 | INFO     | __main__:time_operation:36 - 
np.asarray array info:
2024-12-13 17:42:54.431 | INFO     | __main__:time_operation:37 - Writeable: True
2024-12-13 17:42:54.432 | INFO     | __main__:time_operation:38 - Shape: (2316, 3970, 3)
2024-12-13 17:42:54.440 | INFO     | __main__:time_operation:45 - 
np.asarray timing:
2024-12-13 17:42:54.441 | INFO     | __main